# Importamos librerías y cargamos datos con Pandas en 3 df
## Eliminamos filas con datos nulos
## Imprimimos head y describe para ver el aspecto de los df

In [34]:
import numpy as np
import pandas as pd

inventories = pd.read_csv("inventories.csv")
inventories = inventories.dropna()
print(inventories.head())
# print(inventories.describe())

sales = pd.read_csv("sales.csv")
sales = sales.dropna()
print(sales.head())
# print(sales.describe())

satisfaction = pd.read_csv("satisfaction.csv")
satisfaction = satisfaction.dropna()
print(satisfaction.head())
# print(satisfaction.describe())


   ID_Tienda    Producto  Stock_Disponible Fecha_Actualización
0          1  Producto A                50          2023-01-05
1          1  Producto B                40          2023-01-06
2          2  Producto A                60          2023-01-07
3          2  Producto C                45          2023-01-08
4          3  Producto A                30          2023-01-09
   ID_Tienda    Producto  Cantidad_Vendida  Precio_Unitario Fecha_Venta
0          1  Producto A                20              100  2023-01-05
1          1  Producto B                15              200  2023-01-06
2          2  Producto A                30              100  2023-01-07
3          2  Producto C                25              300  2023-01-08
4          3  Producto A                10              100  2023-01-09
   ID_Tienda  Satisfacción_Promedio Fecha_Evaluación
0          1                     85       2023-01-15
1          2                     90       2023-01-15
2          3                   

## Exploración de datos:
### Calcula las ventas totales por producto y por tienda.
### Calcula los ingresos totales por tienda, multiplicando la cantidad vendida por el precio unitario.
### Genera un resumen estadístico de las ventas utilizando el método describe() para obtener la media, mediana y otras métricas clave.
### Si los productos están clasificados por categorías, calcula el promedio de ventas por tienda y categoría de productos.
### Utiliza groupby() en Pandas para calcular las ventas totales por tienda o por categoría.

In [35]:
# Ventas totales por producto
ventas_por_producto = sales.groupby('Producto')['Cantidad_Vendida'].sum()
print("Ventas totales por producto:")
print(ventas_por_producto)

print("\n")

# Ventas totales por tienda
ventas_por_tienda = sales.groupby('ID_Tienda')['Cantidad_Vendida'].sum()
print("Ventas totales por tienda:")
print(ventas_por_tienda)

Ventas totales por producto:
Producto
Producto A    85
Producto B    75
Producto C    90
Name: Cantidad_Vendida, dtype: int64


Ventas totales por tienda:
ID_Tienda
1    35
2    55
3    50
4    60
5    50
Name: Cantidad_Vendida, dtype: int64


In [36]:
# Ingresos totales por tienda
sales['Ingresos'] = sales['Cantidad_Vendida'] * sales['Precio_Unitario']
ingresos_por_tienda = sales.groupby('ID_Tienda')['Ingresos'].sum()
print("Ingresos totales por tienda:")
print(ingresos_por_tienda)

# Resumen estadístico de ventas
print(sales.describe())

Ingresos totales por tienda:
ID_Tienda
1     5000
2    10500
3     9000
4    13000
5    13000
Name: Ingresos, dtype: int64
       ID_Tienda  Cantidad_Vendida  Precio_Unitario      Ingresos
count  10.000000         10.000000        10.000000     10.000000
mean    3.000000         25.000000       190.000000   5050.000000
std     1.490712          9.128709        87.559504   3361.960407
min     1.000000         10.000000       100.000000   1000.000000
25%     2.000000         20.000000       100.000000   2625.000000
50%     3.000000         25.000000       200.000000   3500.000000
75%     4.000000         30.000000       275.000000   7875.000000
max     5.000000         40.000000       300.000000  10500.000000


In [37]:
# promedio de ventas por tienda y categoría de productos
ventas_promedio = sales.groupby(['ID_Tienda', 'Producto'])['Cantidad_Vendida'].mean()
print("\n")
print("Promedio de ventas por tienda y categoría de productos:")
print(ventas_promedio)




Promedio de ventas por tienda y categoría de productos:
ID_Tienda  Producto  
1          Producto A    20.0
           Producto B    15.0
2          Producto A    30.0
           Producto C    25.0
3          Producto A    10.0
           Producto B    40.0
4          Producto A    25.0
           Producto C    35.0
5          Producto B    20.0
           Producto C    30.0
Name: Cantidad_Vendida, dtype: float64


## Análisis de inventarios (Pandas)
### Calcula la rotación de inventarios para cada tienda. Esto se hace dividiendo las ventas totales por el stock disponible de cada producto.
### Almacena los resultados en una nueva columna dentro del DataFrame de inventarios.
### Filtra y muestra las tiendas con niveles críticos de inventario, es decir, aquellas tiendas donde el porcentaje de productos vendidos sea menor al 10% del stock disponible.
### Utiliza groupby() y operaciones matemáticas para calcular la rotación de inventarios.
### Aplica filtros con Pandas para identificar las tiendas con niveles críticos.

In [38]:
# Rotación de inventarios por tienda
rotacion_inventarios = sales.groupby(['ID_Tienda', 'Producto'])['Cantidad_Vendida'].sum() / inventories.groupby(['ID_Tienda', 'Producto'])['Stock_Disponible'].sum()
print(rotacion_inventarios)
inventories = inventories.merge(rotacion_inventarios.rename('Rotacion_Inventarios'), on=['ID_Tienda', 'Producto'], how='left')
print(inventories.head())

ID_Tienda  Producto  
1          Producto A    0.400000
           Producto B    0.375000
2          Producto A    0.500000
           Producto C    0.555556
3          Producto A    0.333333
           Producto B    0.500000
4          Producto A    0.500000
           Producto C    0.500000
5          Producto B    0.500000
           Producto C    0.500000
dtype: float64
   ID_Tienda    Producto  Stock_Disponible Fecha_Actualización  \
0          1  Producto A                50          2023-01-05   
1          1  Producto B                40          2023-01-06   
2          2  Producto A                60          2023-01-07   
3          2  Producto C                45          2023-01-08   
4          3  Producto A                30          2023-01-09   

   Rotacion_Inventarios  
0              0.400000  
1              0.375000  
2              0.500000  
3              0.555556  
4              0.333333  


In [39]:
# Filtro 1: Baja rotación (ventas < 10% del stock)
tiendas_baja_rotacion = inventories[inventories['Rotacion_Inventarios'] < 0.10]
print("Tiendas con baja rotación (ventas < 10% del stock):")
print(tiendas_baja_rotacion[['ID_Tienda', 'Producto', 'Stock_Disponible', 'Rotacion_Inventarios']])

print("\n")

# Filtro 2: Stock crítico (stock < 10% de ventas, es decir rotación > 10)
tiendas_stock_critico = inventories[inventories['Stock_Disponible'] < 0.10 * sales['Cantidad_Vendida']]
print("Tiendas con stock crítico (stock < 10% de ventas):")
print(tiendas_stock_critico[['ID_Tienda', 'Producto', 'Stock_Disponible', 'Rotacion_Inventarios']])

Tiendas con baja rotación (ventas < 10% del stock):
Empty DataFrame
Columns: [ID_Tienda, Producto, Stock_Disponible, Rotacion_Inventarios]
Index: []


Tiendas con stock crítico (stock < 10% de ventas):
Empty DataFrame
Columns: [ID_Tienda, Producto, Stock_Disponible, Rotacion_Inventarios]
Index: []


## Satisfacción del cliente (Pandas)
### Realiza un análisis de la satisfacción de los clientes en cada tienda. Relaciona estos datos con el rendimiento de las ventas.
### Filtra las tiendas con niveles bajos de satisfacción (< 60%) y haz recomendaciones para mejorar el rendimiento de estas tiendas.

In [41]:
# Análisis de satisfacción relacionado con ventas
# Combinar satisfacción con ingresos por tienda
analisis_satisfaccion = satisfaction.merge(
    ingresos_por_tienda.reset_index().rename(columns={'Ingresos': 'Ingresos_Totales'}),
    on='ID_Tienda'
)
print("Satisfacción vs Rendimiento de Ventas:")
print(analisis_satisfaccion)

print("\n")

# Filtrar tiendas con baja satisfacción (< 60%)
tiendas_baja_satisfaccion = analisis_satisfaccion[analisis_satisfaccion['Satisfacción_Promedio'] < 60]
print("Tiendas con baja satisfacción (< 60%):")
print(tiendas_baja_satisfaccion)

print("\n")

# Recomendaciones
print("RECOMENDACIONES para tiendas con baja satisfacción:")
for _, tienda in tiendas_baja_satisfaccion.iterrows():
    print(f"\nTienda {tienda['ID_Tienda']} (Satisfacción: {tienda['Satisfacción_Promedio']}%):")
    print("  - Revisar calidad del servicio al cliente")
    print("  - Evaluar disponibilidad de productos")
    print("  - Implementar encuestas de seguimiento")
    print("  - Capacitar al personal en atención al cliente")

Satisfacción vs Rendimiento de Ventas:
   ID_Tienda  Satisfacción_Promedio Fecha_Evaluación  Ingresos_Totales
0          1                     85       2023-01-15              5000
1          2                     90       2023-01-15             10500
2          3                     70       2023-01-15              9000
3          4                     65       2023-01-15             13000
4          5                     55       2023-01-15             13000


Tiendas con baja satisfacción (< 60%):
   ID_Tienda  Satisfacción_Promedio Fecha_Evaluación  Ingresos_Totales
4          5                     55       2023-01-15             13000


RECOMENDACIONES para tiendas con baja satisfacción:

Tienda 5 (Satisfacción: 55%):
  - Revisar calidad del servicio al cliente
  - Evaluar disponibilidad de productos
  - Implementar encuestas de seguimiento
  - Capacitar al personal en atención al cliente



## Operaciones con Numpy
### Usar Numpy para realizar los siguientes cálculos sobre las ventas:
#### - Mediana de las ventas totales.
#### - Desviación estándar de las ventas totales.
### Para los cálculos, convierte la columna Total_Ventas del DataFrame de Pandas a un array de Numpy usando .to_numpy (o .values si lo prefieres).
### Genera arrays aleatorios utilizando la biblioteca Numpy para simular proyecciones de ventas futuras.
### Usa el módulo de aleatoriedad de Numpy y asegúrate de establecer una semilla (seed) para obtener resultados reproducibles.

In [46]:
# Convertir columna Ingresos a array de Numpy
array_ingresos = sales['Ingresos'].to_numpy()
print("Array de ingresos:", array_ingresos)

# Mediana de las ventas totales
mediana = np.median(array_ingresos)
print(f"\nMediana de ingresos: {mediana}")

# Desviación estándar de las ventas totales
desviacion_std = np.std(array_ingresos)
print(f"Desviación estándar de ingresos: {desviacion_std}")

# Proyecciones de ventas futuras con arrays aleatorios
# Establecer semilla para reproducibilidad
np.random.seed(42)

# Simular proyecciones basadas en media y desviación estándar actuales
media_ingresos = np.mean(array_ingresos)
proyecciones = np.random.normal(loc=media_ingresos, scale=desviacion_std, size=12)  # 12 meses
print(f"\nProyecciones de ventas futuras (12 meses):")
print(proyecciones)

print(f"\nResumen de proyecciones:")
print(f"  Proyección mínima: {np.min(proyecciones):.2f}")
print(f"  Proyección máxima: {np.max(proyecciones):.2f}")
print(f"  Proyección promedio: {np.mean(proyecciones):.2f}")

Array de ingresos: [ 2000  3000  3000  7500  1000  8000 10500  2500  4000  9000]

Mediana de ingresos: 3500.0
Desviación estándar de ingresos: 3189.4356867634124

Proyecciones de ventas futuras (12 meses):
[ 6634.23784573  4609.01490364  7115.76093733  9907.60577603
  4303.18287048  4303.23523392 10086.79771077  7497.68371242
  3552.64163948  6780.46036522  3571.95907267  3564.58490358]

Resumen de proyecciones:
  Proyección mínima: 3552.64
  Proyección máxima: 10086.80
  Proyección promedio: 5993.93
